In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import files

# --- STEP 1: FOLDER SETUP & DATA UPLOAD ---
base_dir = 'MyShoesDataset'
brands = ['Nike', 'Adidas', 'Puma']
IMG_SIZE = (224, 224)

# Create folders
for brand in brands:
    os.makedirs(os.path.join(base_dir, brand), exist_ok=True)

# Dialogue boxes for each brand
for brand in brands:
    print(f"\n--- 👟 UPLOAD 15-20 IMAGES FOR: {brand.upper()} ---")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dest_path = os.path.join(base_dir, brand, filename)
        os.rename(filename, dest_path)
    print(f"✅ Saved to {brand} folder.")

# --- STEP 2: DATA LOADING & PREPARATION ---
print("\n⚙️ Preparing datasets...")
# We use 20% of your uploads for validation to check accuracy
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=4  # Small batch size is better for small datasets
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=4
)

class_names = train_ds.class_names

# --- STEP 3: MODEL BUILDING & TRAINING ---
print("\n🧠 Building and training model for 10 epochs...")
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    # Data Augmentation helps when you only have a few images
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train for exactly 10 epochs as requested
model.fit(train_ds, validation_data=val_ds, epochs=10)
print("✅ Training Complete!")

# --- STEP 4: FINAL TESTING DIALOGUE ---
print("\n" + "="*40)
print("✨ TEST TIME! UPLOAD A FINAL SAMPLE IMAGE ✨")
print("="*40)

test_upload = files.upload()

for fn in test_upload.keys():
    img = tf.keras.utils.load_img(fn, target_size=IMG_SIZE)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    # Using argmax to find the brand with the highest percentage
    result_idx = np.argmax(predictions[0])
    predicted_brand = class_names[result_idx]
    confidence = 100 * np.max(predictions[0])

    print("\n" + "-"*30)
    print(f"👟 RESULT: This is {predicted_brand} shoes!")
    print(f"📊 Confidence: {confidence:.2f}%")
    print("-" * 30)

In [ ]:
print("\n" + "="*40)
print("✨ FINAL TEST: UPLOAD A SAMPLE IMAGE ✨")
print("="*40)

test_upload = files.upload()

for fn in test_upload.keys():
    img = tf.keras.utils.load_img(fn, target_size=IMG_SIZE)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    # Using argmax to find the brand with the highest percentage
    result_idx = np.argmax(predictions[0])
    predicted_brand = class_names[result_idx]
    confidence = 100 * np.max(predictions[0])

    print("\n" + "-"*30)
    print(f"👟 RESULT: This is {predicted_brand} shoes!")
    print(f"📊 Confidence: {confidence:.2f}%")
    print("-" * 30)